In [2]:
#import pandas as pd
#import string
from matplotlib import rc

fontsize = 12

# Make default font serif
rc('font', family=b'serif', size=fontsize)
# Hide legend frame, change size, make only one scatter marker show up
rc('legend', borderpad=0.,labelspacing=0.25, fontsize=(fontsize - 1.), frameon=False, numpoints=1)
# Default line width (e.g. for plot()) should be thinner than 1 point
rc('patch', linewidth=0.75)
rc('lines', linewidth=0.75)

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


# BW1484 Core I Output

In [3]:
k_ref = 1.00020

### Load and parse the file

In [4]:
import re
import os
import numpy as np

# name of the file containing the output we want to process; 
# figure out how to make this a command line option later
out_file='bw1484_small.out' #$1
full_out_file = os.path.join('/home/slayer/Dropbox/Software/Exnihilo/Calculations/bw1484/',
                            out_file)
if os.path.isfile(full_out_file):
    f = open(full_out_file, 'r')
else:
    raise IOError('could not find file', full_out_file)

# Pairs of output variables that are in the header / problem specification
keys = ['problem_name', 'num cells I', 'num cells J', 'num cells K', 'cells', 'mesh ',
        'num_blocks_i', 'num_blocks_j', 'num_z_blocks', 'blocks', 'num_groups', 
        'num upscatter groups', 'num_sets', 'domains', 'Pn_order', 'azimuthals_octant', 
        'polars_octant', 'eq_set', 'DoF', 'tolerance', 'L2_tolerance', 'k_tolerance',
        'up_tol', 'eigen_solver', 'mg_preconditioner', 'relax_count', 'num_v_cycles', 
        'relax_weight', 'ml_Pn', 'ml_az', 'ml_oct']
vals = {key: None for key in keys}


time_keys = ['procs-per-node-x', 'procs-per-node-y', 'walltime']
time_vals = {key: None for key in time_keys}

# Iteration information
key_map = {'fxd_iters' : 'Iterations =',# 'fxd_res' : '[ 0 ] =', 
           'eigen_iters' : 'Eigenvalue Iteration   ', 
           'rel_err_k' : 'relative error in k-eff of ', 
           'abs_err_k' : 'absolute error in k-eff of '}
iter_dict = {'Iterations =' : [],# '[ 0 ] =' : [],
             'Eigenvalue Iteration   ' : [], 'relative error in k-eff of ' : [],
             'absolute error in k-eff of ' : []}

# set flags and keys to use for items with duplicate entries
ml_flag = False
ml_keys = ['Pn_order', 'azimuthals_octant', 'polars_octant']
ml_doubles = {key: None for key in ml_keys}
ml_cnt = 0
up_db = False
doubles = {'tolerance' : None}

for line in f:
    
    # check for the duplicate items before processing the next stuff
    if ml_flag == True: 
        if ml_cnt != 3:
            for ml_key in ml_doubles:
                if re.search(ml_key, line):
                    if ml_doubles[ml_key] == None:
                        data = line.split(ml_key)
                        ml_doubles[ml_key] = data[1].strip() 
                        ml_cnt = ml_cnt + 1
        else:
            ml_flag = False
    if up_db == True:
        if re.search('tolerance', line):
            if doubles['tolerance'] == None:
                data = line.split('tolerance')
                doubles['tolerance'] = data[1].strip() 
                up_db = False
    
    # Information in the header
    for key in vals: 
        # check for things to set the flags
        if re.search('multilevel_quad_db', line):
            ml_flag = True
        if re.search('upscatter_db', line):
            up_db = True   
        # now look for the rest of the keys
        if re.search(key, line):
            data = line.split(key)
            vals[key] = data[1].strip()  
            
    # Time and CUDA information
    for key in time_vals:
        if re.search(key, line):
            data = line.split(key)
            num = data[1].strip().split()
            time_vals[key] = num[0].strip()
            
    # Iteration information
    for key in iter_dict:
        if re.search(key, line):
            data = line.split(key)
            num = data[1].strip().split()
            iter_dict[key].append(num[0].strip())
        
# calculate things we want in the spreadsheet
vals['cells'] = int(vals['num cells I']) * int(vals['num cells J']) * int(vals['num cells K'])
vals['blocks'] = int(vals['num_blocks_i']) * int(vals['num_blocks_j'])
vals['domains'] = int(vals['blocks']) * int(vals['num_sets'])
# set doubles
vals['ml_Pn'] = ml_doubles['Pn_order']
vals['ml_az'] = ml_doubles['azimuthals_octant']
vals['ml_oct'] = ml_doubles['polars_octant']
vals['up_tol'] = doubles['tolerance']
  

### print information to csv

In [12]:
import csv

# Reorient the values in iter_dict and put into a dictionary with nice names
#iter_dict_final = {}
#for key in key_map:
#    iter_dict_final[key] = np.reshape(iter_dict[key_map[key]], 
#                                      (len(iter_dict[key_map[key]]), 1))
iter_keys = ['eigen_iters', 'rel_err_k', 'abs_err_k', 'fxd_iters']

key_map = {'fxd_iters' : 'Iterations =',# 'fxd_res' : '[ 0 ] =', 
           'eigen_iters' : 'Eigenvalue Iteration   ', 
           'rel_err_k' : 'relative error in k-eff of ', 
           'abs_err_k' : 'absolute error in k-eff of '}

with open('output.csv', 'wb') as csvfile:
    # write the header / problem specification information
    writer = csv.DictWriter(csvfile, fieldnames = keys, dialect='excel', 
                            restval='',extrasaction='ignore')
    #if not csv.Sniffer().has_header(csvfile.read(1024)):
    writer.writeheader()
    writer.writerow(vals)
    
    # Now write timing information
    writer = csv.DictWriter(csvfile, fieldnames = time_keys, dialect='excel', 
                            restval='',extrasaction='ignore')
    writer.writeheader()
    writer.writerow(time_vals)
    
    # And iteration information
    writer = csv.DictWriter(csvfile, fieldnames = iter_keys, dialect='excel', 
                            restval='',extrasaction='ignore')
    writer.writeheader()
    length = len(iter_dict[key_map[iter_keys[0]]])
    for row_num in range(length):
        # get a new dictionary that will contain one row of data at each new row
        tmp_dict = {}
        for key in iter_keys:
            tmp_dict[key] = iter_dict[key_map[key]][row_num]
        writer.writerow(tmp_dict)
    